In [1]:
# Defining imports and constants. 

# Imports
# Required imports are at it's place where it is needed to be used. However here, there should be a list of all 
# imports so that it will be possible to use those just by runnig the 1st shell.
import librosa
import soundfile as sf
import numpy as np
import os
import shutil
from itertools import product
import pyrubberband

# Constants
END = '\n\n'
SAMPLE_RATE = 384000
# SAMPLE_RATE = 8000


In [2]:
# Global variable

# All english alphabets
# a b c-> d e f g h i j k l m n o p q- r s t u v w- x- y- z

# Stands for Vyanjanavarna set.
vv_set = [
    'k', 'kh', 'g', 'gh', 'n2',
    'ch', 'chh', 'j', 'jh',
    't', 'th', 'd', 'dh',
    't2', 'th2', 'd2', 'dh2',
    'p', 'ph', 'b', 'bh',
    
    'f', 'v', 'z',
    
    'y', 'w', 'q', 'x',
    
    # A part of the vv is pvv too.
    'r', 'l', 'sh', 's', 'h',
    'n', 'm',
]

pvv_set = [
    'r', 'l', 'sh', 's',
    'n', 'm',
    'g', 'j', 'd', 'd2', 'b',
    'gh', 'jh', 'dh', 'dh2', 'bh',
]

# Stands for Swaravarna set.
sv_set = [
    'a', 'a2', 'i', 'u', 'e', 'o',
#     'ae', 'ao',
    # All these mixed sv are not required in this set. Insted we are going to collect these from real large text
    # corpuses.
#     'aa2', 'ai', 'au', 'ae', 'ao',
#     'a2a', 'a2i', 'a2u', 'a2e', 'a2o',
#     'ia', 'ia2', 'iu', 'ie', 'io',
#     'ua', 'ua2', 'ui', 'ue', 'uo',
#     'ea', 'ea2', 'ei', 'eu', 'eo',
#     'oa', 'oa2', 'oi', 'ou', 'oe',
    
    # Extra Mappings. 
    'aa'
]

# Stands for Punctuation set. 
p_set = [
    ',', '.', '?', '!'
]

char_set = sv_set + vv_set + p_set

sv_set_large = [
    # Here we are going to append all the new sv those are collected from large text corpus. 
]


### Main Pipeline

In [3]:
# This shell is responsible for taking a word as input and converting it to it's character list.

# input: 'ankush'
# output: ['a', 'n', 'k', 'u', 'sh']

def word_to_chars(word, char_1, char_2, char_3, char_4):
    # We will be taking 4 different lists.
    word_to_char_list = []

    i = 0
    final_index = len(word)
    
    # Writing the word_to_char_list formatter script.
    while i < final_index:
        # Check if the character is in the char_4.
        try:
            inst_char = word[i : i+4]
            if inst_char in char_4:
                word_to_char_list.append(inst_char)
                i = i + 4
                continue
        except:
            pass

        # Check if the character is in the char_3.
        try:
            inst_char = word[i : i+3]
            if inst_char in char_3:
                word_to_char_list.append(inst_char)
                i = i + 3
                continue
        except:
            pass

        # Check if the character is in the char_2.
        try:
            inst_char = word[i : i+2]
            if inst_char in char_2:
                if inst_char == 'aa':
                    word_to_char_list.append('a2')
                else:
                    word_to_char_list.append(inst_char)
                i = i + 2
                continue
        except:
            pass

        # Check if the character is in the char_1.
        try:
            inst_char = word[i : i+1]
            if inst_char in char_1:
                word_to_char_list.append(inst_char)
                i = i + 1
                continue
            else:
                # If the character was not found anywhere append '_na_' for it.
                word_to_char_list.append('_na_')
                i = i + 1
        except:
            pass
        
    return word_to_char_list

# Using the function.
word = 'raekt2esh'

char_1 = []
char_2 = []
char_3 = []
char_4 = []

for char in char_set:
    if len(char) == 4:
        char_4.append(char)
    if len(char) == 3:
        char_3.append(char)
    if len(char) == 2:
        char_2.append(char)
    if len(char) == 1:
        char_1.append(char)

# print(char_1, end=END)
# print(char_2, end=END)
# print(char_3, end=END)
# print(char_4, end=END)

word_to_char_list = word_to_chars(word, char_1, char_2, char_3, char_4)
print(word_to_char_list)


['r', 'a', 'e', 'k', 't2', 'e', 'sh']


In [4]:
# This shell is responsible for merging the swaravarnas given a word_to_char_list.

def word_to_connected_chars(word_to_char_list):
    word_to_connected_char_list = []

    inst_sv = ''

    for i in range(len(word_to_char_list)):
        inst_char = word_to_char_list[i]

        if inst_char in sv_set:
            inst_sv = inst_sv + inst_char

        if inst_char in vv_set:
            if inst_sv != '':
                word_to_connected_char_list.append(inst_sv)
                # Also append it in the sv_set_large.
                sv_set_large.append(inst_sv)
                inst_sv = ''

            word_to_connected_char_list.append(inst_char)

        if i == len(word_to_char_list) - 1:
            if inst_sv != '':
                word_to_connected_char_list.append(inst_sv)
                # Also append it in sv_set_large
                sv_set_large.append(inst_sv)
                
        # Append any punctuation if it's present.
        if inst_char in [',', '.', '?', '!']:
            word_to_connected_char_list.append(inst_char)
                
    return word_to_connected_char_list
        
word = 'raekt2esh'
word_to_char_list = word_to_chars(word, char_1, char_2, char_3, char_4)
print(word_to_char_list)

word_to_connected_char_list = word_to_connected_chars(word_to_char_list)
print(word_to_connected_char_list)


['r', 'a', 'e', 'k', 't2', 'e', 'sh']
['r', 'ae', 'k', 't2', 'e', 'sh']


In [5]:
# This shell is responsible for creating the fluctuated character list out of the main character list.

def char_fluctuator(word_to_char_list):
    chars = word_to_connected_char_list
    # print(chars, end=END)

    char_flct = []

    for i in range(len(chars)):
        char = chars[i]
        if char in vv_set:
            try:
                char_after = chars[i + 1]
                char_before = chars[i - 1]
                if char_after in sv_set_large:
                    char_flct.append(char + '_s')
                    
                elif (char_before in sv_set_large) and (i != 0):
                    char_flct.append(char + '_e')
                    char_flct.append('_g_')
                else:
                    char_flct.append(char + '_s')
            except:
                pass
            if i == len(chars)-1:
                char_flct.append(char + '_e')
        elif char in sv_set_large:
            char_flct.append(char)

    # print(char_flct, end=END)

    # Now reversing the char_flct
    char_flct.reverse()
    # print(char_flct, end=END)

    char_flct_final = []

    j = 0
    for i in range(len(char_flct)):
        char = char_flct[i]

        if char in sv_set_large:
            # Check if ending SV.
            is_ending_sv = True
            for k in range(i+1, len(char_flct)):
                char_nexts = char_flct[k]
                if char_nexts in sv_set_large:
                    is_ending_sv = False

            if j == 0:
                char_flct_final.append(char + '_e')
                j = j + 1

            elif j > 1 and is_ending_sv == True:
                char_flct_final.append(char + '_s')

            elif j > 0:
                char_flct_final.append(char + '_m')
                j = j + 1
        else:
            char_flct_final.append(char)

    char_flct_final.reverse()

    # print(char_flct_final, end=END)

    # Now it's time to handle the pvv.
    char_flct_list_altimate = []

    for i in range(len(char_flct_final)):
        char_flct = char_flct_final[i]
        char = char_flct.split('_')[0]
        char_suffix = char_flct.split('_')[1]
        if char in pvv_set and char_suffix == 'e':
            try:
                char_before = char_flct_final[i - 1]
                char_before_suffix = char_before.split('_')[1]
                char_before_pure = char_before.split('_')[0]
                if char_before_pure in sv_set_large:
                    char_flct_list_altimate.append(char_flct + '_' + char_before_suffix)
                else:
                    char_flct_list_altimate.append(char_flct + '_e')
            except:
                pass
        else:
            char_flct_list_altimate.append(char_flct)
            
    print(char_flct_list_altimate, end=END)
    
    # Now it's time to handle the speacial 'h' sound when it comes.
    char_flct_list_altimate_2 = []
        
    for i in range(len(char_flct_list_altimate)):
        char_flct = char_flct_list_altimate[i]
        char = char_flct.split('_')[0]
        char_flct_suffix = char_flct.split('_')[1]
        
        if (char_flct_suffix == 's') and (char[-1] == 'h' or char[-2:] == 'h2') and (char != 'ch'):
            if char == 'h':
                char_split_1 = 'h'
            elif char[-2:] == 'h2':
                char_split_1 = char[0: len(char)-2] + '2'
            else:
                char_split_1 = char[0 : len(char)-1]
            
            char_next = char_flct_list_altimate[i + 1]
            char_next_pure = char_next.split('_')[0]
            
            if (char_next_pure in sv_set) or (char_next_pure in sv_set_large):
                first_letter = char_next_pure[0]
                needed_h = 'h_s_' + first_letter
                if char != 'h':
                    char_flct_list_altimate_2.append(char_split_1 + '_s')
                    char_flct_list_altimate_2.append(needed_h)
                else:
                    char_flct_list_altimate_2.append(needed_h)
            else:
                char_flct_list_altimate_2.append(char_split_1)
                char_flct_list_altimate_2.append('h_s_a')
            
        elif char_flct == 'h_e':
            before_char = char_flct_list_altimate[i - 1]
            before_char_pure = before_char.split('_')[0]
            if (before_char_pure in sv_set) or (before_char_pure in sv_set_large):
                last_letter = before_char_pure[-1]
                needed_h = 'h_e_' + last_letter
                char_flct_list_altimate_2.append(needed_h)
            else:
                char_flct_list_altimate_2.append(char_flct)
        else:
            char_flct_list_altimate_2.append(char_flct)
    
    return char_flct_list_altimate_2



# Using the function.
word = 'trartotpro'

word_to_char_list = word_to_chars(word, char_1, char_2, char_3, char_4)
print(word_to_char_list, end=END)

word_to_connected_char_list = word_to_connected_chars(word_to_char_list)
print(word_to_connected_char_list, end=END)

char_flct_list = char_fluctuator(word_to_connected_char_list)
print(char_flct_list)


['t', 'r', 'a', 'r', 't', 'o', 't', 'p', 'r', 'o']

['t', 'r', 'a', 'r', 't', 'o', 't', 'p', 'r', 'o']

['t_s', 'r_s', 'a_s', 'r_e_s', '_g_', 't_s', 'o_m', 't_e', '_g_', 'p_s', 'r_s', 'o_e']

['t_s', 'r_s', 'a_s', 'r_e_s', '_g_', 't_s', 'o_m', 't_e', '_g_', 'p_s', 'r_s', 'o_e']


In [6]:
# This shell is responsible for converting char_flct_list to it's speech form.

# input: ['a_s', 'n_e', '_g_', 'k_s', 'u_m', 'sh_e']
# output: 'word.wav'

def char_flct_to_speech(char_flct_list, noise, vv_set_name, sv_set_name, pvv_set_name, vv_volume, sv_volume, yukt_gap=0.2):
    # Beofore putting any gaps we need to import the noise and crop the part as gap as per the gap time
    # specified. 
    noise_ar, sr = librosa.load('data/noise_set/{}.wav'.format(noise), sr=SAMPLE_RATE)
    
    _g_ = noise_ar[: int(yukt_gap * SAMPLE_RATE)]

    speech_ar = np.array([])
    
    concatenation_case = []

    for i in range(len(char_flct_list)):
        inst_char = char_flct_list[i]
        inst_char_pure = inst_char.split('_')[0]
        inst_char_suffix = inst_char.split('_')[1]

        if (inst_char_pure in vv_set):
            if inst_char_pure in pvv_set and inst_char_suffix == 'e' :
                inst_chunk_ar, inst_sr = librosa.load('data/sv_sets/{}/{}/{}'.format(sv_set_name, pvv_set_name, inst_char + '.wav'), sr = SAMPLE_RATE) 
            else:
                inst_chunk_ar, inst_sr = librosa.load('data/vv_sets/{}/{}'.format(vv_set_name, inst_char + '.wav'), sr=SAMPLE_RATE)
                inst_chunk_ar = inst_chunk_ar * vv_volume
            speech_ar = np.concatenate([speech_ar, inst_chunk_ar])
            concatenation_case.append(inst_char)
        
        if inst_char == '_g_':
            inst_chunk_ar = _g_
            speech_ar = np.concatenate([speech_ar, inst_chunk_ar])
            concatenation_case.append(inst_char)
        
        if inst_char_pure in sv_set_large:
            # Handling the concatenation of the sv is a bit different.
            try:
                char_after = char_flct_list[i + 1]
                char_after_pure = char_after.split('_')[0]
                char_after_pure_suffix = char_after.split('_')[1]
                
                if char_after_pure in vv_set:
                    if char_after_pure in pvv_set and char_after_pure_suffix == 'e':
                        char_after_ar, inst_sr = librosa.load('data/sv_sets/{}/{}/{}'.format(sv_set_name, pvv_set_name, char_after + '.wav'), sr = SAMPLE_RATE)
                    else:
                        char_after_ar, inst_sr = librosa.load('data/vv_sets/{}/{}'.format(vv_set_name, char_after + '.wav'), sr=SAMPLE_RATE)
                        
                    inst_chunk_ar, inst_sr = librosa.load('data/sv_sets/{}/{}'.format(sv_set_name, inst_char + '.wav'), sr=SAMPLE_RATE)
                    
                    char_after_len = len(char_after_ar)
                    inst_chunk_len = len(inst_chunk_ar)
                    
                    # To see if the it's taking unsusual concatenation.
                    # print(inst_chunk_len, char_after_len)
                    
                    if inst_chunk_len >char_after_len:
                        inst_chunk_ar_appendable = inst_chunk_ar[: inst_chunk_len - char_after_len]
                        speech_ar = np.concatenate([speech_ar, inst_chunk_ar_appendable])
                        concatenation_case.append(inst_char)
                    else:
                        concatenation_case.append('unsual_concatenation_susp')
                else:
                    inst_chunk_ar, inst_sr = librosa.load('data/sv_sets/{}/{}'.format(sv_set_name, inst_char + '.wav'), sr=SAMPLE_RATE)
                    inst_chunk_ar = inst_chunk_ar * sv_volume
                    speech_ar = np.concatenate([speech_ar, inst_chunk_ar])
                    concatenation_case.append(inst_char)
            except:
                inst_chunk_ar, inst_sr = librosa.load('data/sv_sets/{}/{}'.format(sv_set_name, inst_char + '.wav'), sr=SAMPLE_RATE)
                inst_chunk_ar = inst_chunk_ar * sv_volume
                speech_ar = np.concatenate([speech_ar, inst_chunk_ar])
#                 concatenation_case.append(inst_char)
                
    print('concatenation_case:', concatenation_case)
    return speech_ar

# vv_l = 'k'
# word = vv_l + 'a' + vv_l + 'aa' + vv_l + 'i' + vv_l + 'u' + vv_l + 'e' + vv_l + 'o'

word = 'muskaan'

word_to_char_list = word_to_chars(word, char_1, char_2, char_3, char_4)
print('character_list:', word_to_char_list, end=END)

word_to_connected_char_list = word_to_connected_chars(word_to_char_list)
print('connected_character_list:', word_to_connected_char_list, end=END)

char_flct_list = char_fluctuator(word_to_connected_char_list)
print('char_flct_list', char_flct_list, end=END)

noise = 'noise_n2'
vv_set_name = 'vv_set_n2_t2'
sv_set_name = 'sv_set_n2_t2_s2_revised'
pvv_set_name = 'pvv_set_n2_t2_s2_revised'
yukt_gap = 0.001
vv_volume = 1
sv_volume = 1

# It become true if all the characters needed for a word pronunciation exist in a folder. 
is_ready = False

existed_vv = os.listdir('data/vv_sets/{}/'.format(vv_set_name))
existed_vv = [vv.split('.')[0] for vv in existed_vv]
# print(existed_vv, end=END)

existed_sv = os.listdir('data/sv_sets/{}/'.format(sv_set_name))
existed_sv = [sv.split('.')[0] for sv in existed_sv]
# print(existed_sv, end=END)

existed_pvv = os.listdir('data/sv_sets/{}/{}/'.format(sv_set_name, pvv_set_name))
existed_pvv = [pvv.split('.')[0] for pvv in existed_pvv]
not_exist_chars = []

for char in char_flct_list:
    if char == '_g_':
        pass
    elif char in existed_vv:
        pass
    elif char in existed_sv:
        pass
    elif char in existed_pvv:
        pass
    else:
        not_exist_chars.append(char)
        
print('not_existed_charaters:', not_exist_chars, end=END)

if len(not_exist_chars) == 0:
    is_ready = True

# Only execute the audio concatenation if all the chunks are available.
if is_ready == True:
    speech_ar = char_flct_to_speech(char_flct_list, noise, vv_set_name, sv_set_name, pvv_set_name, vv_volume, sv_volume, yukt_gap)
    
    sf.write('outputs/word.wav', speech_ar, SAMPLE_RATE, format='wav')
    
    # Now chaning the pitch of the speech.
#     speed = 0.5
#     path_to_word = '/home/sainilayam/Desktop/dev/project_21/tts_rnd/outputs/word.wav'
#     path_to_output = '/home/sainilayam/Desktop/dev/project_21/tts_rnd/outputs/word_sc.wav'
#     cmd = 'ffmpeg -i {} -filter:a "atempo={}" -vn {} -y'.format(path_to_word, speed, path_to_output)
#     os.system(cmd)
    
    # Now chaning the speed of the speech, as per expectation.
    expected_length = 0.7
    actual_length = librosa.get_duration(speech_ar, sr=SAMPLE_RATE)
    speed_index = actual_length / expected_length
    
    path_to_word = '/home/sainilayam/Desktop/dev/project_21/tts_rnd/outputs/word.wav'
    path_to_output = '/home/sainilayam/Desktop/dev/project_21/tts_rnd/outputs/word_sc.wav'
    cmd = 'ffmpeg -i {} -filter:a "atempo={}" -vn {} -y'.format(path_to_word, speed_index, path_to_output)
    os.system(cmd)
    
    # Printing out the message.
    print('Word saved with file name \'word.wav\' in the outputs folder.')


character_list: ['m', 'u', 's', 'k', 'a2', 'n']

connected_character_list: ['m', 'u', 's', 'k', 'a2', 'n']

['m_s', 'u_m', 's_e_m', '_g_', 'k_s', 'a2_e', 'n_e_e']

char_flct_list ['m_s', 'u_m', 's_e_m', '_g_', 'k_s', 'a2_e', 'n_e_e']

not_existed_charaters: []

concatenation_case: ['m_s', 'u_m', 's_e_m', '_g_', 'k_s', 'a2_e', 'n_e_e']


/tmp/ipykernel_4889/2481814172.py:146: FutureWarning: Pass y=[-0.00094604 -0.00094604 -0.00100708 ...  0.00085449  0.00076294
  0.00085449] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  actual_length = librosa.get_duration(speech_ar, sr=SAMPLE_RATE)
ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg 

Word saved with file name 'word.wav' in the outputs folder.


size=     526kB time=00:00:00.68 bitrate=6306.0kbits/s speed=8.76x    
video:0kB audio:525kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.018959%


### So this point is a break over the whole TTS pipeline. If we only want to pronounce variables, then all we need to do is to work up to here. 

In [7]:
# This shell is responsible for taking a sentence as input and convert it to it's speech form. 

# Now here we are going to write a function to pronouce a sentence. 
def sentence_to_speech(sentence, noise, word_gap, comma_gap):
    word_list = sentence.split(' ')
    
    noise_ar, sr = librosa.load('data/noise_set/{}.wav'.format(noise), sr=SAMPLE_RATE)

    _wg_ = noise_ar[: int(word_gap * SAMPLE_RATE)]
    _cg_ = noise_ar[: int(comma_gap * SAMPLE_RATE)]

    sentence_ar = np.array([])
    
    # Adding a little extra part before the ar.
    sentence_ar = np.concatenate([sentence_ar, noise_ar[: int(0.4 * SAMPLE_RATE)]])
    
    # Adding a little bit of gap at the beginning of the clip. 
    sentence_ar = np.concatenate([sentence_ar, _wg_])
    
    for word in word_list:
        if word == ',':
            sentence_ar = np.concatenate([sentence_ar, _cg_])
        else:
            word_to_char_list = word_to_chars(word, char_1, char_2, char_3, char_4)
            word_to_connected_char_list = word_to_connected_chars(word_to_char_list)
            char_flct_list = char_fluctuator(word_to_connected_char_list)
            word_ar = char_flct_to_speech(char_flct_list, noise, vv_set_name, sv_set_name, vv_volume, sv_volume, yukt_gap)
            sentence_ar = np.concatenate([sentence_ar, word_ar, _wg_])
    
    # Adding a little extra part at the end of the clip.
    sentence_ar = np.concatenate([sentence_ar, noise_ar[: int(0.4 * SAMPLE_RATE)]])
    
    return sentence_ar

sentence = 'gaghinkugh'
# sentence = 't2um hama2re dost2ho.' 

noise = 'noise_n2'
vv_set_name = 'vv_set_n2_t2'
sv_set_name = 'sv_set_n2_t2_s2_revised'
pvv_set_name = 'pvv_set_n2_t2_s2_revised'

yukt_gap = 0.02

# Repeatitiong of vv
# vv_repeat = int()
# This thing will be managed by ration implemenation. 

# This thing will be managed with accent transfer with starting middle and ending sounds.
# If Swara comes at the end then fade out.
# sv_fadeout = float()

vv_volume = 0.4
sv_volume = 1

word_gap = 0.00
comma_gap = 0.00

sentence_ar = sentence_to_speech(sentence, noise, word_gap, comma_gap)
# Implementing a sentence level time stretching algorithm.
# sentence_ar = librosa.effects.time_stretch(sentence_ar, rate=1.5)

sf.write('outputs/sentence.wav', sentence_ar, SAMPLE_RATE)

# Printing out the status message.
print('Sentence saved with file name \'sentence.wav\' in the outputs folder.')
    

/home/sainilayam/personal/dev/venvs/tf/lib/python3.10/site-packages/librosa/util/decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)


['m_s', 'u_m', 's_e_m', '_g_', 'k_s', 'a2_e', 'n_e_e']



FileNotFoundError: [Errno 2] No such file or directory: 'data/sv_sets/sv_set_n2_t2_s2_revised/0.4/s_e_m.wav'

### Extra  Features

In [ ]:
# This shell is responsible for taking all the Swaravarnas from a sv_set_name folder and create _s, _m, _e 
# sounds from the main file and save them all in the same folder.

def create_psd_flct_sv(sv_set_name):
#     sv_set_name = 'sv_set_n3_t1_s1'
    sv_set_path = 'data/sv_sets/' + sv_set_name + '/'

    file_list = os.listdir(sv_set_path)

    for i in range(len(file_list)):
        inst_file = file_list[i]
        
        # Check if the Varna exists in sv_set to prevent remake of files those are already processed.
        inst_file_varna = inst_file.split('.')[0]
        
        if inst_file_varna in sv_set:
            inst_file_source_path = sv_set_path + inst_file

            inst_file_destiantion_path = sv_set_path + inst_file.split('.')[0] + '_s.wav'
            shutil.copy(inst_file_source_path, inst_file_destiantion_path)

            inst_file_destiantion_path = sv_set_path + inst_file.split('.')[0] + '_m.wav'
            shutil.copy(inst_file_source_path, inst_file_destiantion_path)

            inst_file_destiantion_path = sv_set_path + inst_file.split('.')[0] + '_e.wav'
            shutil.copy(inst_file_source_path, inst_file_destiantion_path)

    # Writing out the message status.
    print('All files with the name _s, _m, _e created at the destination folder.')
    
# Using the function.
sv_set_name = 'sv_set_n6_t1_s1'
create_psd_flct_sv(sv_set_name)


In [ ]:
# This shell is responsible for taking all the Vyanjana Varnas from a vv_set_name folder and create _s, _e 
# sounds from the main file and save them all in the same folder.

def create_psd_flct_vv(vv_set_name):
#     vv_set_name = 'vv_set_n1_t1'
    vv_set_path = 'data/vv_sets/' + vv_set_name + '/'

    file_list = os.listdir(vv_set_path)

    for i in range(len(file_list)):
        inst_file = file_list[i]
        
        # Check if the Varna exists in sv_set to prevent remake of files those are already processed.
        inst_file_varna = inst_file.split('.')[0].split('_')[0]
        
        if inst_file_varna in vv_set:
            inst_file_source_path = vv_set_path + inst_file

#             inst_file_destiantion_path = sv_set_path + inst_file.split('.')[0] + '_s.wav'
#             shutil.copy(inst_file_source_path, inst_file_destiantion_path)

#             inst_file_destiantion_path = sv_set_path + inst_file.split('.')[0] + '_m.wav'
#             shutil.copy(inst_file_source_path, inst_file_destiantion_path)

            inst_file_destiantion_path = vv_set_path + inst_file.split('.')[0].split('_')[0] + '_e.wav'
            shutil.copy(inst_file_source_path, inst_file_destiantion_path)

    # Writing out the message status.
    print('All files with the name _s, _e created at the destination folder.')
    
# Using the function.
vv_set_name = 'vv_set_n1_t1'
create_psd_flct_vv(vv_set_name)


In [ ]:
# This shell is responsible for creating a k Replced sentence from a normal sentence.

# input: 'di em a2e fainans me a2pka2 sua2gat2 hae.'
# output: 'ki e_na_ a2e _na_aikakk _na_e a2kka2 kua2kak kae.'
# further output: 'i e_na_ a2e _na_aia _na_e a2a2 ua2a ae.'

# The reason we are doing this is to pronouce the whole sentece with 'k' as Vyanjana Varna and then we can 
# pronouce the sentence without 'k' to get the pure Swaravarna sounds with the way these should be pronouced. 

# Then we need a script that will only keep the Swaravarnas, spaces, ',', '.', '?', '!'  and removes all other 
# Vyanjavarnas with 'k'.

def k_replaced_sentence_maker(text):
    text_ar = text.split(' ')
    
    text_k_ar = [] 

    for i in range(len(text_ar)):
        inst_word = text_ar[i]
        inst_word_to_char_ar = word_to_chars(inst_word, char_1, char_2, char_3, char_4)

        for j in range(len(inst_word_to_char_ar)):
            inst_char = inst_word_to_char_ar[j]
            if inst_char in vv_set:
                inst_word_to_char_ar[j] = 'k'

        inst_word = ''.join(inst_word_to_char_ar)
        text_k_ar.append(inst_word)

    text_k = ' '.join(text_k_ar)
    
    return text_k

# sentence = 'di em a2e fainans me a2pka2 sua2gat2 hae.'
sentence = 'diepha2epe a2pka2 sua2gat2hae.'

print(sentence, end=END)

sentence_k = k_replaced_sentence_maker(sentence)
print(sentence_k, end=END)

sentence_sv = sentence_k.replace('k', '')
print(sentence_sv)


In [ ]:
# This shell is reposible for taking input a Vyanjana Varna and test with all different Swaravarnas to see check
# the quality of that Vyanjanavarna crop.

# input: 'k'
# output: A clip with the sounds 'ka', 'ka2', 'ki', 'ku', 'ke', 'ko'

def create_sv_vv_test_clip(vv, sv_list):
    # vv = 'k_s'
    # sv_list = ['a', 'a2', 'i', 'u', 'e', 'o']

    noise = 'noise_n2'
    vv_set = 'vv_set_n2_t1'
    sv_set = 'sv_set_n2_t1_s1'

    # Importing the noise.
    noise_path = 'data/noise_set/' + noise + '.wav'
    noise_ar, sr = librosa.load(noise_path, sr=SAMPLE_RATE)
    
    gap_ar = noise_ar[: int(0.05 * SAMPLE_RATE)]
    
    # Improting the vv.
    vv_path = 'data/vv_sets/' + vv_set + '/' + vv + '.wav'
    vv_ar, sr = librosa.load(vv_path, sr=SAMPLE_RATE) 

    vv_sv_ar = np.array([])
    # Adding a 0.4 second gap at the beginning of the ar.
    vv_sv_ar = np.concatenate([vv_sv_ar, noise_ar[: int(0.4 * SAMPLE_RATE)]])

    # Now we need to write the concatenation loop.
    for i in range(len(sv_list)):
        inst_sv = sv_list[i]
        inst_sv_path = 'data/sv_sets/' + sv_set + '/' + inst_sv + '.wav'
        inst_sv_ar, sr = librosa.load(inst_sv_path, sr=SAMPLE_RATE)
        
        # Detecting if the Vyanjanavarna is starting or the ending sound.
        inst_sign = vv.split('_')[1]
        
        if inst_sign == 's':
            vv_sv_ar = np.concatenate([vv_sv_ar, vv_ar, inst_sv_ar, noise_ar[: int(0.4 * SAMPLE_RATE)]])
        if inst_sign == 'e':
            vv_sv_ar = np.concatenate([vv_sv_ar, inst_sv_ar, vv_ar, noise_ar[: int(0.4 * SAMPLE_RATE)]])
        
    # Now writing the ar.
    sf.write('outputs/vv_sv.wav', vv_sv_ar, SAMPLE_RATE)
    # Showing the status message.
    print('Outputs are generated at the output folder with name \'vv_sv.wav\'.')

# Using the function.
vv = 'k_s'
sv_list = ['a', 'a2', 'i', 'u', 'e', 'o']
create_sv_vv_test_clip(vv, sv_list)


In [ ]:
# This shell is responsible for outputting all the underlying information of a sentence w.r.t how the TTS
# pipeline is working.

# input: ''
# output_1: ''
# output_2: ''
# output_3: ''

def get_pipeline_info(sentence):
    # sentence = 'di em a2e fainans me a2pka2 sua2gat2 hae.'
    print('sentence:', sentence, end=END)

    word_list = sentence.split(' ') 

    sentence_to_connected_char_list = []

    for word in word_list:
        inst_word_to_char_list = word_to_chars(word, char_1, char_2, char_3, char_4)
        inst_word_to_connected_char_list = word_to_connected_chars(inst_word_to_char_list)
        sentence_to_connected_char_list.append(inst_word_to_connected_char_list)
    print('sentence_to_chars:',sentence_to_connected_char_list, end=END)
    
    sentence_char_flct_list = []

    for char_list in sentence_to_connected_char_list:
        inst_char_flct_list = char_fluctuator(char_list)
        sentence_char_flct_list.append(inst_char_flct_list)
    print('sentence_to_char_flct:', sentence_char_flct_list, end=END)

    sentece_vv_flct_list = []

    sentence_sv_flct_list = []
    sentence_sv_list = []

    for char_flct_list in sentence_char_flct_list:

        for char in char_flct_list:
            inst_char_pure = char.split('_')[0]

            if inst_char_pure in vv_set:
                sentece_vv_flct_list.append(char)

            if inst_char_pure in sv_set_large:
                sentence_sv_flct_list.append(char)
                sentence_sv_list.append(inst_char_pure)

    sentece_vv_flct_list_unique = list(dict.fromkeys(sentece_vv_flct_list))

    sentence_sv_flct_list_unique = list(dict.fromkeys(sentence_sv_flct_list))
    sentence_sv_list_unique = list(dict.fromkeys(sentence_sv_list))

    print('sentence_vv_flct_unique:', sentece_vv_flct_list_unique, end=END)
    print('length_of_vv_flct_uniqe:', len(sentence_sv_flct_list_unique), end=END)
    print('setnce_sv_flct_unique:', sentence_sv_flct_list_unique, end=END)
    print('sentce_sv_unique:', sentence_sv_list_unique, end=END)
    print('length_of_sv_unique:', len(sentence_sv_list_unique), end=END)
    
    # Returning sentence_to_char_flct.
    return sentence_char_flct_list
    
# sentence = 'di em a2e fainans me a2pka2 sua2gat2 hae.'
# sentence = 'diepha2epe a2pka2 sua2gat2hae.'
sentence = 'a2pjobhiho, achheho.'

get_pipeline_info(sentence)


### Procedure to build the TTS.

In [ ]:
# 1. First take your whole text. 
# 2. Write it in the way it should be according to the script notation that this TTS follow. Also feel free to 
# update the notations. Remember not to mess up with any previous notation.
# 3. Use proper commas after each phrases properly.
# 4. Use the information section to get what are different vv and sv needed for the TTS to run.
# 5. For different persons, take vvs in different tones.
# 6. For different styles go with different sv sounds. The vv set used here is going to be of the same tone 
# in the tone it should be 
# 7. For the sv sounds, crop it from real word pronunciations.
# 8. Once you get a _s, _m, _e sound of one length sv, you can just mimic exactly the same for other vvs as
# well.


In [ ]:
# This shell is responsible for taking a list of sentences as input and give information of those sentnces
# iteratively. You can also get the infromation of the whole text.

# All Setnences taken in to consideration.
# --------
sentences = [
#     'a2pka2 ham, kia2, mad2at2, kar sakt2e, haen?',
#     'bat2aie, a2p kaese ho?',
#     'hama2re pa2s, isse jia2da2 bata2ne ke lie, kuchh nahi hae',
#     'paribart2an sansa2rka2 niam hae.',
#     'di em a2e fa2inans me, a2pka2, sua2gat hae.',
    'ham sab, ek haen.',
    't2um hama2re, dost2 ho.',
#     'ra2z a2nkhen t2eri, sab baea2n kar rahi.',
#     'a2pki lon ki ra2shi, pacha2s rupae hae.',
]
# --------

# Convert the comma separated parts of the sentece to phrases by making thsese as words.
sentence_index = 0

whole_text = ''
for sentence in sentences:
    whole_text = whole_text + sentence + ','

# --------
# Swtich to get information iteratively or as a whole. Switch here by doing commenting and uncommenting. 
# sentence = sentences[sentence_index]
sentence = whole_text
# --------

print('sentence:', sentence, end=END)

sentence_space_removed = sentence.replace(' ', '')
sentence_comma_splitted = sentence_space_removed.split(',')

# Creating the sentence with proper phrases.
sentence_phrased = ' '.join(sentence_comma_splitted)
print('sentece_phrased:', sentence_phrased, end=END)

# Getting the sentence info.
print('Sentece Informations: ', end='\n\n')
sentence_to_char_flct = get_pipeline_info(sentence_phrased)


In [ ]:
# This shell is responsible for taking a sv_- as input and show a word that contains it, so that we can pronouce
# that word and crop it from there. 

sv_flct = 'a_s'

for i in range(len(sentence_to_char_flct)):
    inst_char_flct = sentence_to_char_flct[i]
    
    if sv_flct in inst_char_flct:
        try:
            inst_char_flct.remove('_g_')
        except:
            pass
        
        phrase = ''
        for char_flct in inst_char_flct:
            if char_flct != sv_flct:
                inst_char_pure = char_flct.split('_')[0]
                phrase = phrase + inst_char_pure
            else:
                phrase = phrase + '-' + char_flct + '-'
        
        print(inst_char_flct)
        
        print(phrase)
        
        break
    

In [ ]:
# This shell is responsible for changing the sample rate of any wav file to a different sample rate.

file_path = '/home/sainilayam/personal/inst/ranjan.wav'
target_file_name = '/home/sainilayam/personal/inst/{}'.format('resampled.wav')
target_sample_rate = 8000

clip_ar, sr = librosa.load(file_path, sr=target_sample_rate)
sf.write(target_file_name, clip_ar, target_sample_rate)

print('Resampled file created at target location.')


### Research

In [ ]:
# This shell is responsible for finding a the length of a wav file give the path to it. 

path_to_wav = '/home/sainilayam/personal/dev/project_21/tts_rnd/data/sv_sets/sv_set_n2_t2_s2/pvv_set_n2_t2_s2/n_e_e.wav'

ar, sr = librosa.load(path_to_wav, sr=SAMPLE_RATE)
ar_len = librosa.get_duration(ar, sr=SAMPLE_RATE)

print(ar_len)


In [ ]:
# This shell is responsible for aligning all the similar type of varnas in equal time frame. 

# len_dict = {
#     'sv_s': 0.31,
#     'sv_m': 0.34,
#     'sv_e': 0.47,
    
#     'pvv_e_s': 0.25,
#     'pvv_e_m': 0.29,
#     'pvv_e_e': 0.33
# }

len_dict = {
    'sv_s': 0.31,
    'sv_m': 0.28,
    'sv_e': 0.47,
    
    'pvv_e_s': 0.25,
    'pvv_e_m': 0.24,
    'pvv_e_e': 0.33
}


sv_path =  '/home/sainilayam/personal/dev/project_21/tts_rnd/data/sv_sets/sv_set_n2_t2_s2/'
pvv_path = '/home/sainilayam/personal/dev/project_21/tts_rnd/data/sv_sets/sv_set_n2_t2_s2/' + 'pvv_set_n2_t2_s2/'

def grider(len_dict, sv_path, pvv_path):
    files_1 = os.listdir(sv_path)
    files_2 = os.listdir(pvv_path)

    files = files_1 + files_2
    files = [file for file in files if file.split('_')[0] != 'pvv']

    print(files, end='\n\n')
    print(len(files))

    for i in range(len(files)):
        file = files[i]
        file_name_pure = file.split('.')[0]
        varna_name = file_name_pure.split('_')[0]

        if (varna_name in sv_set) or (varna_name in sv_set_large):  
            varna_suffix = file_name_pure.split('_')[-1]
            if varna_suffix == 's':
                expected_len = len_dict['sv_s']
            if varna_suffix == 'm':
                expected_len = len_dict['sv_m']
            if varna_suffix == 'e':
                expected_len = len_dict['sv_e']

            ar, sr = librosa.load('{}{}'.format(sv_path, file), SAMPLE_RATE)
            ar_len = librosa.get_duration(ar, sr=SAMPLE_RATE)

            speed_index = ar_len / expected_len
            print(speed_index)

            path_to_varna = sv_path + file
            path_to_output = '/home/sainilayam/personal/dev/project_21/tts_rnd/data/sv_sets/sv_set_revised/'  + file

            print(path_to_output)

            cmd = 'ffmpeg -i {} -filter:a "atempo={}" -vn {} -y'.format(path_to_varna, speed_index, path_to_output)

            os.system(cmd)

        if varna_name in vv_set:
            varna_suffix = file_name_pure.split('_')[-1]
            if varna_suffix == 's':
                expected_len = len_dict['pvv_e_s']
            if varna_suffix == 'm':
                expected_len = len_dict['pvv_e_m']
            if varna_suffix == 'e':
                expected_len = len_dict['pvv_e_e']

            ar, sr = librosa.load('{}{}'.format(pvv_path, file), SAMPLE_RATE)
            ar_len = librosa.get_duration(ar, sr=SAMPLE_RATE)

            speed_index = ar_len / expected_len
            print(speed_index)

            path_to_varna = pvv_path + file
            path_to_output = '/home/sainilayam/personal/dev/project_21/tts_rnd/data/sv_sets/sv_set_revised/pvv_set_revised/'  + file

            print(path_to_output)

            cmd = 'ffmpeg -i {} -filter:a "atempo={}" -vn {} -y'.format(path_to_varna, speed_index, path_to_output)

            os.system(cmd)

    print('All svs and pvv_e sounds are being properly grided and are placed in proper folders.')

# Using the function.
grider(len_dict, sv_path, pvv_path)



In [ ]:
# This shell if responsible for making one _e sound to _e_s, _e_m and _e_e
path_to_folder = '/home/sainilayam/personal/dev/project_21/tts_rnd/data/sv_sets/sound_factory/'
path_to_output_folder = '/home/sainilayam/personal/dev/project_21/tts_rnd/data/sv_sets/sound_factory_revised/'

files = os.listdir(path_to_folder)

for i in range(len(files)):
    file = files[i]
    file_path = path_to_folder + file
    
    new_file_name = file.split('.')[0] + '_s' + '.wav'
    destination_file_path = path_to_output_folder + new_file_name
    shutil.copy(file_path, destination_file_path)
    
    new_file_name = file.split('.')[0] + '_m' + '.wav'
    destination_file_path = path_to_output_folder + new_file_name
    shutil.copy(file_path, destination_file_path)
    
    new_file_name = file.split('.')[0] + '_e' + '.wav'
    destination_file_path = path_to_output_folder + new_file_name
    shutil.copy(file_path, destination_file_path)
    
    

In [8]:
# This shell is responsible for cropping out a bunch of words from a given sound clip.

# Creating array that will hold all the words. 
words_1 = [
    'january', 'february', 'march', 'april', 'may', 'june', 'july', 'august', 'september', 'october', 
    'november', 'december',
]

words_2 = [
    'sunya', 'ek', 'do', 'tin', 'char', 'panch', 'chhe', 'saat', 'aath', 'nao', 'das', 'igyara', 'bara',
    'tera', 'chaoda', 'pandra', 'sola', 'satra', 'athara', 'unis', 'bis', 
]

words_3 = [
    'ikis', 'bais', 'teis', 'chaobis', 'pachis', 'chhabis', 'satais', 'athais', 'unatis', 'tis', 'ikatis',
    'batis', 'taentis', 'chaotis', 'paentis', 'chhatis', 'saentis', 'athtis', 'untalis', 'chalis',
]

words_4 = [
    'iktalis', 'bayalis', 'tiyalis', 'chaowalis', 'paentalis', 'chhialis', 'saentalis', 'athtalis', 
    'unachas', 'pachas', 'ikyaban', 'baaban', 'tirepan', 'chaoban', 'pachpan', 'chhapan', 'sataban', 'athaban',
    'unsath', 'sath',
]

words_5 = [
    'iksath', 'basath', 'tesath', 'chaosath', 'paensath', 'chhiesath', 'sadsath', 'adsath', 'unhtar', 'satar',
    'ikatar', 'bahatar', 'tihetar', 'chaohatar', 'pachatar', 'chhihatar', 'satattar', 'athattar', 'unyasi', 
    'asi',
]

words_6 = [
    'ikyasi', 'bayasi', 'tiyasi', 'chaowasi', 'pachasi', 'chhiasi', 'satasi', 'athasi', 'inyanbe', 'nabe',
    'ikyanbe', 'biyanbe', 'tiyanbe', 'chowanbe', 'pachanbe', 'chhiyanbe', 'sataanbe', 'athanbe', 'ninyanbe',
]

words_7 = [
    'sao', 'hazar', 'lakh', 'karod',
]

words_8 = [
    'car_loan', 'house_loan'
]

words_9 = [
    'abhishek'
]

words_list = words_1

# The cropper scirpt.
ar_original, sr = librosa.load('words_1.wav', sr=SAMPLE_RATE)
ar, sr = librosa.load('words_1.wav', sr=48000)
split_indices = librosa.effects.split(ar, top_db=4, frame_length=int(48000 * 0.02), hop_length=int(48000*0.5))
print(len(split_indices), end=END)
print(split_indices, end=END)

for i in range(len(words_list)):
    try:
        word_name = words_list[i]
        file_name = word_name + '.wav'
        path = 'chunks/' + file_name

        start_index = split_indices[i][0]
        start_index_original = int(start_index / 48000) * SAMPLE_RATE

        end_index = split_indices[i][1]
        end_index_original = int(end_index / 48000) * SAMPLE_RATE

        chunk = ar_original[start_index_original : end_index_original]

        sf.write(path, chunk, SAMPLE_RATE)
    except:
        pass
    


2

[[240000 264000]
 [312000 336000]]



In [16]:
# This shell is responsible for converting a whole bunch of audio files to 8000 Hz.

data_factory = 'data_factory/'
data_factory_output = 'data_factory_output/'

files = os.listdir(data_factory)
# print(files)

for i in range(len(files)):
    file = files[i]
    input_path = data_factory + file
    output_path = data_factory_output + file
    
    ar, sr = librosa.load(input_path, sr=8000)
    sf.write(output_path, ar, 8000)
    
print('All files are converted to 8000 Hz.')


All files are converted to 8000 Hz.
